# Data Upload & transform data

In [1]:
import pandas as pd
import sqlite3 as sq

url_1 = 'https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_movies.csv'
url_2 = 'https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_credits.csv'

movies = pd.read_csv(url_1, sep=',')
credits = pd.read_csv(url_2, sep=',')
movies.to_csv('/workspaces/JLL_KNN_ML/data/raw/movies.csv', index=False)
credits.to_csv('/workspaces/JLL_KNN_ML/data/raw/credits.csv', index=False)


***Create a SQL database***

In [2]:
con = sq.connect("../data/raw/movies_database.db")

movies.to_sql("movies_table", con, if_exists = "replace", index = False)
credits.to_sql("credits_table", con, if_exists = "replace", index = False)

4803

***Join different data***

In [3]:
query = """
SELECT *
FROM movies_table
INNER JOIN credits_table
USING (title)
"""
df_movies = pd.read_sql_query(query, con)
con.close()

***Import to pandas***

In [4]:
df_movies.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."


***Cleaning data***

In [5]:
columns = ['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']
df_movies = df_movies[columns]

* Select information   

In [6]:
import json

df_movies['genres'] = df_movies['genres'].apply(json.loads)
df_movies['genres'] = df_movies['genres'].apply(lambda x: [i['name'] for i in x])
df_movies['keywords'] = df_movies['keywords'].apply(json.loads)
df_movies['keywords'] = df_movies['keywords'].apply(lambda x: [i['name'] for i in x])
df_movies['cast'] = df_movies['cast'].apply(json.loads)
df_movies['cast'] = df_movies['cast'].apply(lambda x: [i['name'] for i in x][:3])
df_movies['crew'] = df_movies['crew'].apply(json.loads)
df_movies['crew'] = df_movies['crew'].apply(lambda x: [i['name'] for i in x if i['job'] == 'Director'])
df_movies['overview'] = df_movies['overview'].apply(lambda x: [x])

* Clear spaces of names

In [7]:
df_movies["cast"] = df_movies["cast"].apply(lambda x: [name.replace(' ', '') for name in x])
df_movies["genres"] = df_movies["genres"].apply(lambda x: [name.replace(' ', '') for name in x])
df_movies["keywords"] = df_movies["keywords"].apply(lambda x: [name.replace(' ', '') for name in x])
df_movies["crew"] = df_movies["crew"].apply(lambda x: [name.replace(' ', '') for name in x])

In [8]:
df_movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In the 22nd century, a paraplegic Marine is d...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain Barbossa, long believed to be dead, h...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]
2,206647,Spectre,[A cryptic message from Bond’s past sends him ...,"[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes]
3,49026,The Dark Knight Rises,[Following the death of District Attorney Harv...,"[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan]
4,49529,John Carter,"[John Carter is a war-weary, former military c...","[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton]


* Create a new column: tags

In [9]:
df_movies["overview"] = df_movies["overview"].apply(lambda x: [str(x)])
df_movies["genres"] = df_movies["genres"].apply(lambda x: [str(genre) for genre in x])
df_movies["keywords"] = df_movies["keywords"].apply(lambda x: [str(keyword) for keyword in x])
df_movies["cast"] = df_movies["cast"].apply(lambda x: [str(actor) for actor in x])
df_movies["crew"] = df_movies["crew"].apply(lambda x: [str(crew_member) for crew_member in x])

df_movies["tags"] = df_movies["overview"] + df_movies["genres"] + df_movies["keywords"] + df_movies["cast"] + df_movies["crew"]
df_movies["tags"] = df_movies["tags"].apply(lambda x: ",".join(x).replace(",", " "))

df_movies.drop(columns = ["genres", "keywords", "cast", "crew", "overview"], inplace = True)


In [10]:
df_movies.head()

,movie_id,title,tags
0,19995,Avatar,['In the 22nd century a paraplegic Marine is ...
1,285,Pirates of the Caribbean: At World's End,['Captain Barbossa long believed to be dead ...
2,206647,Spectre,['A cryptic message from Bond’s past sends him...
3,49026,The Dark Knight Rises,"[""Following the death of District Attorney Har..."
4,49529,John Carter,"[""John Carter is a war-weary former military ..."


* Saving the data

In [11]:
df_movies.to_csv('../data/processed/data_movies_clean.csv', index=False)
con = sq.connect('../data/raw/movies_database.db')
movies.to_sql('data_movies', con, if_exists = 'replace', index = False)

4803

# KNN Model

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df_movies["tags"])

model = NearestNeighbors(n_neighbors = 6, algorithm = "brute", metric = "cosine")
model.fit(tfidf_matrix)

def get_movie_recommendations(movie_title):
    movie_index = df_movies[df_movies["title"] == movie_title].index[0]
    distances, indices = model.kneighbors(tfidf_matrix[movie_index])
    similar_movies = [(df_movies["title"][i], distances[0][j]) for j, i in enumerate(indices[0])]
    return similar_movies[1:]

input_movie = "Rocky"
recommendations = get_movie_recommendations(input_movie)
print("Film recommendations '{}'".format(input_movie))
for movie, distance in recommendations:
    print("- Film: {}".format(movie))

Film recommendations 'Rocky'
- Film: Rocky Balboa
- Film: Creed
- Film: Cinderella Man
- Film: The Fighter
- Film: Undisputed
